<h2 style="text-align: center;">Prvi seminarski praktični deo</h2>

<h3 style="text-align: center;">Ames Housing Dataset</h3>

### Učitavanje i osnovna analiza podataka
U ovom koraku se učitavaju podaci iz dataset-a Ames Housing i vrši se osnovna analiza strukture podataka, tipovi atributa i ukupnog broja instanci.

In [1]:
import pandas as pd
df = pd.read_csv("AmesHousing.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


### Kreiranje binarne ciljne promenjive
Prodajna cena nekretnine se trasformiše u binarnu promenljivu na osnovu medijane vrednosti.

In [2]:
median_price = df['SalePrice'].median()
df['PriceCategory'] = df['SalePrice'].apply(lambda x: 1 if x > median_price else 0)
x = df.drop(['SalePrice', 'PriceCategory'], axis = 1)
y = df['PriceCategory']

### Baseline model bez transformacije podataka

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

categorical_cols = x.select_dtypes(include = 'object').columns
le = LabelEncoder()

for col in categorical_cols:
    x[col] = le.fit_transform(x[col].astype(str))

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

imputer = SimpleImputer (strategy="mean")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

baseline_model = LogisticRegression(max_iter = 1000)
baseline_model.fit(X_train, y_train)
y_pred_baseline = baseline_model.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Accuracy: {baseline_accuracy:.3f}")

Accuracy: 0.869


### Standardizacija numeričkih atributa
Numerički atributi se trasnformišu kako bi imali srednju vrednost 0 i standardnu devijaciju 1.

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

### Treniranje modela nakon transformcije

In [7]:
scaled_model = LogisticRegression(max_iter=1000)
scaled_model.fit(x_train_scaled, y_train)

y_pred_scaled = scaled_model.predict(x_test_scaled)
scaled_accuracy = accuracy_score(y_test, y_pred_scaled)
print(scaled_accuracy)

0.9419795221843004
